In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import scipy
import scipy.stats as stats
import sklearn.impute as simpleImputer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler,StandardScaler,RobustScaler,PolynomialFeatures,QuantileTransformer
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
import plotly.express as px

import acquire
import acq
import wrangle_zillow
import env

In [2]:
df0 = wrangle_zillow.get_zillow_single_unit_2017_cluster()
df0.shape

/Users/geary.shenck/codeup-data-science/clustering-exercises/wrangle_zillow.py:16: DtypeWarning: Columns (64) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("zillow_single_fam_sold_2017_cluster.csv", index_col = 0)


(77574, 68)

In [3]:
cols_missing, rows_missing = wrangle_zillow.summarize(df0)


DataFrame head: 

        id  parcelid  airconditioningtypeid  architecturalstyletypeid  \
0  1727539  14297519                    NaN                       NaN   
1  1387261  17052889                    NaN                       NaN   
2    11677  14186244                    NaN                       NaN   
3  2288172  12177905                    NaN                       NaN   
4  1970746  10887214                    1.0                       NaN   

   basementsqft  bathroomcnt  bedroomcnt  buildingclasstypeid  \
0           NaN          3.5         4.0                  NaN   
1           NaN          1.0         2.0                  NaN   
2           NaN          2.0         3.0                  NaN   
3           NaN          3.0         4.0                  NaN   
4           NaN          3.0         3.0                  NaN   

   buildingqualitytypeid  calculatedbathnbr  ...  censustractandblock  \
0                    NaN                3.5  ...         6.059063e+13   
1     

In [4]:
cols_missing[cols_missing.percent_rows_missing > 75]

,num_rows_missing,percent_rows_missing
architecturalstyletypeid,77368,99.734447
basementsqft,77524,99.935545
buildingclasstypeid,77559,99.980664
decktypeid,76960,99.208498
finishedfloor1squarefeet,71539,92.220332
finishedsquarefeet13,77532,99.945858
finishedsquarefeet15,74547,96.097919
finishedsquarefeet50,71539,92.220332
finishedsquarefeet6,77188,99.502411
fireplacecnt,69287,89.317297


In [5]:
rows_missing[rows_missing.percent_cols_missing > 50]

,num_cols_missing,percent_cols_missing,count
12,35,51.470588,5163
13,36,52.941176,5793
14,37,54.411765,3630
15,38,55.882353,1932
16,39,57.352941,285
17,40,58.823529,230
18,41,60.294118,29
19,42,61.764706,26
20,43,63.235294,29
21,44,64.705882,79


In [6]:
df0.columns

Index(['id', 'parcelid', 'airconditioningtypeid', 'architecturalstyletypeid',
       'basementsqft', 'bathroomcnt', 'bedroomcnt', 'buildingclasstypeid',
       'buildingqualitytypeid', 'calculatedbathnbr', 'decktypeid',
       'finishedfloor1squarefeet', 'calculatedfinishedsquarefeet',
       'finishedsquarefeet12', 'finishedsquarefeet13', 'finishedsquarefeet15',
       'finishedsquarefeet50', 'finishedsquarefeet6', 'fips', 'fireplacecnt',
       'fullbathcnt', 'garagecarcnt', 'garagetotalsqft', 'hashottuborspa',
       'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet',
       'poolcnt', 'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7',
       'propertycountylandusecode', 'propertylandusetypeid',
       'propertyzoningdesc', 'rawcensustractandblock', 'regionidcity',
       'regionidcounty', 'regionidneighborhood', 'regionidzip', 'roomcnt',
       'storytypeid', 'threequarterbathnbr', 'typeconstructiontypeid',
       'unitcnt', 'yardbuildingsqft17', 'yardb

In [7]:
df = wrangle_zillow.data_prep_drop(df0, cols_to_remove=[
                            "roomcnt","heatingorsystemtypeid","airconditioningtypeid",\
                                "architecturalstyletypeid","buildingclasstypeid","typeconstructiontypeid",\
                                    "storytypeid","propertylandusetypeid"], column_prop_required=.5, row_prop_required=.75)
df.shape

(77418, 31)

In [8]:
df.propertylandusedesc.value_counts()

Single Family Residential                     52373
Condominium                                   19283
Duplex (2 Units, Any Combination)              2020
Planned Unit Development                       1945
Quadruplex (4 Units, Any Combination)           727
Triplex (3 Units, Any Combination)              539
Cluster Home                                    335
Mobile Home                                      63
Manufactured, Modular, Prefabricated Homes       52
Residential General                              31
Cooperative                                      29
Commercial/Office/Residential Mixed Used         15
Townhouse                                         6
Name: propertylandusedesc, dtype: int64

In [9]:
df = df[(df.propertylandusedesc == "Single Family Residential") | (df.propertylandusedesc == "Condominium")]
df= df[df.unitcnt < 2]
df.shape

(45343, 31)

In [10]:
df.head(5).T

,3,4,6,7,8
id,2288172,1970746,781532,870991,1246926
parcelid,12177905,10887214,12095076,12069064,12790562
bathroomcnt,3.0,3.0,3.0,1.0,3.0
bedroomcnt,4.0,3.0,4.0,2.0,4.0
buildingqualitytypeid,8.0,8.0,9.0,5.0,9.0
calculatedbathnbr,3.0,3.0,3.0,1.0,3.0
calculatedfinishedsquarefeet,2376.0,1312.0,2962.0,738.0,3039.0
finishedsquarefeet12,2376.0,1312.0,2962.0,738.0,3039.0
fips,6037.0,6037.0,6037.0,6037.0,6037.0
fullbathcnt,3.0,3.0,3.0,1.0,3.0


In [11]:
check_for_outliers = ['bathroomcnt', 'bedroomcnt','calculatedbathnbr', 
                        'calculatedfinishedsquarefeet','finishedsquarefeet12','lotsizesquarefeet',
                        'unitcnt','structuretaxvaluedollarcnt', 'taxvaluedollarcnt',
                        'landtaxvaluedollarcnt', 'taxamount']
outliers_df,outliers_list = wrangle_zillow.get_outliers(df[check_for_outliers], 1.5)
outliers_df[outliers_df[outliers_list].sum(axis=1)>0].head(10).T

~~~
bathroomcnt_outlier
count    1204.000000
mean        1.080565
std         0.981620
min         0.500000
25%         0.500000
50%         0.500000
75%         1.500000
max         8.500000
Name: bathroomcnt_outlier, dtype: float64
~~~
bedroomcnt_outlier
count    30.000000
mean      1.466667
std       0.776079
min       1.000000
25%       1.000000
50%       1.000000
75%       2.000000
max       4.000000
Name: bedroomcnt_outlier, dtype: float64
~~~
calculatedbathnbr_outlier
count    1176.000000
mean        1.094388
std         0.989099
min         0.500000
25%         0.500000
50%         0.500000
75%         1.500000
max         8.500000
Name: calculatedbathnbr_outlier, dtype: float64
~~~
calculatedfinishedsquarefeet_outlier
count     2648.000000
mean      1088.166163
std       1442.306304
min          0.500000
25%        248.500000
50%        620.500000
75%       1347.750000
max      18743.500000
Name: calculatedfinishedsquarefeet_outlier, dtype: float64
~~~
finishedsquarefeet12_out

,4,6,9,17,20,24,30,32,33,46
bathroomcnt,3.00,3.00,3.00,3.00,2.00,3.00,3.00,3.000,4.000,3.00
bedroomcnt,3.00,4.00,2.00,3.00,3.00,2.00,4.00,3.000,5.000,2.00
calculatedbathnbr,3.00,3.00,3.00,3.00,2.00,3.00,3.00,3.000,4.000,3.00
calculatedfinishedsquarefeet,1312.00,2962.00,1290.00,1435.00,1092.00,2126.00,2447.00,3641.000,3572.000,1110.00
finishedsquarefeet12,1312.00,2962.00,1290.00,1435.00,1092.00,2126.00,2447.00,3641.000,3572.000,1110.00
lotsizesquarefeet,278581.00,63000.00,54048.00,45171.00,36005.00,221601.00,7911.00,183383.000,20148.000,8500.00
unitcnt,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.000,1.000,1.00
structuretaxvaluedollarcnt,73681.00,276684.00,151303.00,169727.00,140581.00,276641.00,320000.00,833933.000,678493.000,450000.00
taxvaluedollarcnt,119407.00,773303.00,371361.00,259197.00,178646.00,573247.00,1083000.00,1444248.000,1473080.000,563000.00
landtaxvaluedollarcnt,45726.00,496619.00,220058.00,89470.00,38065.00,296606.00,763000.00,610315.000,794587.000,113000.00


In [18]:
outliers_df[outliers_df[outliers_list].sum(axis=1)>0]

,bathroomcnt,bedroomcnt,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,lotsizesquarefeet,unitcnt,structuretaxvaluedollarcnt,taxvaluedollarcnt,landtaxvaluedollarcnt,...,bedroomcnt_outlier,calculatedbathnbr_outlier,calculatedfinishedsquarefeet_outlier,finishedsquarefeet12_outlier,lotsizesquarefeet_outlier,unitcnt_outlier,structuretaxvaluedollarcnt_outlier,taxvaluedollarcnt_outlier,landtaxvaluedollarcnt_outlier,taxamount_outlier
4,3.0,3.0,3.0,1312.0,1312.0,278581.0,1.0,73681.0,119407.0,45726.0,...,0.0,0.0,0.0,0.0,242754.25,0.0,0.0,0.00,0.0,0.000
6,3.0,4.0,3.0,2962.0,2962.0,63000.0,1.0,276684.0,773303.0,496619.0,...,0.0,0.0,0.0,0.0,27173.25,0.0,0.0,0.00,0.0,0.000
9,3.0,2.0,3.0,1290.0,1290.0,54048.0,1.0,151303.0,371361.0,220058.0,...,0.0,0.0,0.0,0.0,18221.25,0.0,0.0,0.00,0.0,0.000
17,3.0,3.0,3.0,1435.0,1435.0,45171.0,1.0,169727.0,259197.0,89470.0,...,0.0,0.0,0.0,0.0,9344.25,0.0,0.0,0.00,0.0,0.000
20,2.0,3.0,2.0,1092.0,1092.0,36005.0,1.0,140581.0,178646.0,38065.0,...,0.0,0.0,0.0,0.0,178.25,0.0,0.0,0.00,0.0,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77562,1.0,2.0,1.0,868.0,868.0,107392.0,1.0,114934.0,259738.0,144804.0,...,0.0,0.0,0.0,0.0,71565.25,0.0,0.0,0.00,0.0,0.000
77565,3.0,3.0,3.0,2200.0,2200.0,7503.0,1.0,226351.0,1414710.0,1188359.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,365504.25,439684.0,3711.145
77566,2.0,2.0,2.0,1261.0,1261.0,57110.0,1.0,131899.0,346788.0,214889.0,...,0.0,0.0,0.0,0.0,21283.25,0.0,0.0,0.00,0.0,0.000
77569,3.0,3.0,3.0,1741.0,1741.0,59487.0,1.0,265000.0,379000.0,114000.0,...,0.0,0.0,0.0,0.0,23660.25,0.0,0.0,0.00,0.0,0.000


In [12]:
df.bathroomcnt.describe()

count    45343.000000
mean         2.206824
std          0.987420
min          0.000000
25%          2.000000
50%          2.000000
75%          3.000000
max         13.000000
Name: bathroomcnt, dtype: float64

In [13]:
df.structuretaxvaluedollarcnt.describe()

count    4.527900e+04
mean     1.839648e+05
std      2.421474e+05
min      1.290000e+02
25%      8.200000e+04
50%      1.314000e+05
75%      2.072725e+05
max      9.164901e+06
Name: structuretaxvaluedollarcnt, dtype: float64

In [14]:
df.shape

(45343, 31)

In [15]:
df.unitcnt.describe()

count    45343.0
mean         1.0
std          0.0
min          1.0
25%          1.0
50%          1.0
75%          1.0
max          1.0
Name: unitcnt, dtype: float64

In [16]:
df.head().T

,3,4,6,7,8
id,2288172,1970746,781532,870991,1246926
parcelid,12177905,10887214,12095076,12069064,12790562
bathroomcnt,3.0,3.0,3.0,1.0,3.0
bedroomcnt,4.0,3.0,4.0,2.0,4.0
buildingqualitytypeid,8.0,8.0,9.0,5.0,9.0
calculatedbathnbr,3.0,3.0,3.0,1.0,3.0
calculatedfinishedsquarefeet,2376.0,1312.0,2962.0,738.0,3039.0
finishedsquarefeet12,2376.0,1312.0,2962.0,738.0,3039.0
fips,6037.0,6037.0,6037.0,6037.0,6037.0
fullbathcnt,3.0,3.0,3.0,1.0,3.0


In [17]:
df[(outliers_df[outliers_list].sum(axis=1) > 0)]

,id,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,...,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,logerror,transactiondate,heatingorsystemdesc,propertylandusedesc
4,1970746,10887214,3.0,3.0,8.0,3.0,1312.0,1312.0,6037.0,3.0,...,73681.0,119407.0,2016.0,45726.0,1533.89,6.037124e+13,0.006940,2017-01-01,Central,Condominium
6,781532,12095076,3.0,4.0,9.0,3.0,2962.0,2962.0,6037.0,3.0,...,276684.0,773303.0,2016.0,496619.0,9516.26,6.037461e+13,-0.001011,2017-01-01,Central,Single Family Residential
9,1585097,11542646,3.0,2.0,8.0,3.0,1290.0,1290.0,6037.0,3.0,...,151303.0,371361.0,2016.0,220058.0,4557.32,6.037275e+13,-0.036763,2017-01-02,Central,Condominium
17,1169121,12045625,3.0,3.0,8.0,3.0,1435.0,1435.0,6037.0,3.0,...,169727.0,259197.0,2016.0,89470.0,2831.48,6.037301e+13,-0.017167,2017-01-02,Central,Condominium
20,2542130,12325145,2.0,3.0,6.0,2.0,1092.0,1092.0,6037.0,2.0,...,140581.0,178646.0,2016.0,38065.0,2391.69,6.037554e+13,0.042463,2017-01-02,Central,Condominium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77562,1089830,10722237,1.0,2.0,8.0,1.0,868.0,868.0,6037.0,1.0,...,114934.0,259738.0,2016.0,144804.0,3225.40,6.037134e+13,0.052630,2017-09-19,Central,Condominium
77565,1684937,12666457,3.0,3.0,10.0,3.0,2200.0,2200.0,6037.0,3.0,...,226351.0,1414710.0,2016.0,1188359.0,16388.30,6.037621e+13,0.028733,2017-09-19,Central,Condominium
77566,1642435,10858613,2.0,2.0,8.0,2.0,1261.0,1261.0,6037.0,2.0,...,131899.0,346788.0,2016.0,214889.0,4231.76,6.037141e+13,-0.075465,2017-09-19,Central,Condominium
77569,2864704,10833991,3.0,3.0,8.0,3.0,1741.0,1741.0,6037.0,3.0,...,265000.0,379000.0,2016.0,114000.0,4685.34,6.037132e+13,-0.002245,2017-09-20,Central,Condominium
